## PROBLEM 1A ###################################

In [ ]:
using JuMP, Gurobi

using DataFrames
using DataFramesMeta

function rob21(X,Y,rho)

    model21 = Model(solver=GurobiSolver(OutputFlag=0))
    n,m = size(X)
    @variable(model21,b[1:m])
    @variable(model21,t)
    @variable(model21,theta)
    @variable(model21,z[1:m])
    @objective(model21, Min, (t + rho*theta))
    @constraint(model21, norm(Y-X*b)<=t)
    @constraint(model21, sum(z[j] for j=1:m)<=theta)
   
    for j=1:m
        @constraint(model21, b[j] <= z[j])
        @constraint(model21, b[j] >= -z[j])
    end 
    
    solve(model21)
    
    getvalue(b)
end
    






rob21 (generic function with 1 method)

In [ ]:
using JuMP, Gurobi
    using DataFrames
    using DataFramesMeta

function rob22(X,Y,rho)
    
    model22 = Model(solver=GurobiSolver(OutputFlag=0))
    n,m = size(X)
    @variable(model22,b[1:m])
    @variable(model22,t)
    @variable(model22,theta)
    @objective(model22, Min, t + rho*theta)
    @constraint(model22, norm(Y - X*b) <= t)
    @constraint(model22, norm(b) <= theta)
   
    
    solve(model22)
    getvalue(b)

end

rob22 (generic function with 1 method)

# PROBLEM 1B ##################################

In [ ]:

using JuMP, Gurobi

using DataFrames
using DataFramesMeta
myData = readtable("C:/Users/subha/Desktop/Housing.csv",header=false)

Y = myData[end]
Y = convert(Array, Y)
X = Matrix(myData[1:end-1])
rho = .5

rob21(X,Y,rho)





Academic license - for non-commercial use only


13-element Array{Float64,1}:
 -0.0904111 
  0.049515  
 -4.85373e-6
  1.08778   
 -2.52278e-7
  5.655     
 -0.00375442
 -0.869761  
  0.175856  
 -0.0104683 
 -0.376258  
  0.0150924 
 -0.447889  

In [ ]:
srand(1)
Y = myData[end]
X = myData[1:end-1]
using MLDataUtils
(train_X, train_Y), (test_X, test_Y) = splitobs(shuffleobs((X, Y)), at=0.7)

train_X = Matrix(train_X)
test_X = Matrix(test_X)
β22 = rob22(train_X, train_Y,.5)
β21 = rob21(train_X, train_Y,.5)


Academic license - for non-commercial use only
Academic license - for non-commercial use only


13-element Array{Float64,1}:
 -0.0988091  
  0.0466247  
 -0.000647395
  0.328779   
 -1.99225e-9 
  5.76715    
 -0.00663176 
 -0.830402   
  0.212765   
 -0.0117071  
 -0.434456   
  0.0161207  
 -0.40873    

In [ ]:
function evaluate(X, y, β)
    norm(y - X * β) / length(y)
end

evaluate (generic function with 1 method)

In [ ]:
function checker(X,y,B)
    norm(y - X * B)
end

checker (generic function with 1 method)

In [ ]:
srand(1)
Y = myData[end]
X = myData[1:end-1]
using MLDataUtils
#split to make training set separate from testing set 
(train_X, train_Y), (test_X, test_Y) = splitobs(shuffleobs((X, Y)), at=0.7)
#
train_X = Matrix(train_X)
test_X = Matrix(test_X)
β22 = rob22(train_X, train_Y,.5)
β21 = rob21(train_X, train_Y,.5)

train_error = evaluate(train_X, train_Y, β22)
test_error = evaluate(test_X, test_Y, β22)


Academic license - for non-commercial use only
Academic license - for non-commercial use only


0.41971582752978476

In [ ]:

#VALIDATION STEP TO GET THE RHO USED TO CHECK THE ERRORS ACROSS MODELS
# Split the training set into training/validation (60%/40%)

(tr_X, tr_y), (vl_X, vl_y) = splitobs((DataFrame(train_X), train_Y), at=0.6)

vl_X = Matrix{Float64}(vl_X)
########### RHO22 VALIDATION TESTING ##############################
best_error22 = Inf
best_rho22 = Inf

for rho in collect(.0001:.5:10) 
#upon initial inspection, noted that the rhos for 21 and 22 
#both on the order of magnitude of 10 X-4, so began refining 
#search for rho to values between 0 and 10 
b = rob22(Matrix(tr_X), tr_y, rho)
error = evaluate(vl_X, vl_y, b)
    if error < best_error22
        best_error22 = error
        best_rho22 = rho
    end
end
best_error22, best_rho22

########################## RHO21 VALIDATION TESTING########################
#upon initial inspection, noted that the rhos for 21 and 22
#both on the order of magnitude of 10 X-4, so began 
#refining search for rho to values between 0 and 10

best_error21 = Inf
best_rho21 = Inf

for rho in collect(.0001:.5:10)
b = rob21(Matrix(tr_X), tr_y, rho)
error = evaluate(vl_X, vl_y, b)
    if error < best_error21
        best_error21 = error
        best_rho21 = rho
    end
end
best_error21, best_rho21
##################################### COMPARE THE ERRORS BETWEEN THE 3 METHODS ##################################
beta21 = rob21(train_X, train_Y,best_rho21)
beta22 = rob22(train_X,train_Y,best_rho22)
OLSbeta= rob22(train_X,train_Y,0)

error21 = checker(test_X,test_Y,beta21)
error22 = checker(test_X,test_Y,beta22)
errorOLS = checker(test_X,test_Y,OLSbeta)




println("BEST RHO FOR ROB21 METHOD = ", best_rho21)
println(" and the error with rho21 on the out of sample test data is ", error21)
println("BEST RHO FOR ROB22 METHOD = ", best_rho22)
println(" and the error with rho21 on the out of sample test data is ", error22)
println("RHO IS 0 for OLS. The error when rho = 0 is ", errorOLS)
println(beta21)
println(beta22)
println(OLSbeta)


   

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

# PROBLEM 2 - SPARSE REGRESSION #########################

In [ ]:
using JuMP, Gurobi

using DataFrames
using DataFramesMeta

mysparseDataY = readtable("C:/Users/subha/Desktop/sparseY.csv",header=false)
mysparseDataX = readtable("C:/Users/subha/Desktop/sparseX.csv",header=false)
TrueBData = readtable("C:/Users/subha/Desktop/sparseB.csv",header=false)

TrueBData=Matrix(TrueBData)


200×1 Array{Int64,2}:
 -1
 -1
  1
  1
  1
 -1
 -1
  1
 -1
  1
 -1
 -1
 -1
  ⋮
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0

## PROBLEM 2A

In [ ]:
function sparseReg(X,Y,k,M)

    modelsp = Model(solver=GurobiSolver(OutputFlag=0))
    m,n = size(X)
    @variable(modelsp,b[1:n])
    @variable(modelsp,z[1:n],Bin)
    @variable(modelsp,t)
    @objective(modelsp,Min,t)
    
    @constraint(modelsp,norm(Y-X*b)<=t)
    
    @constraint(modelsp, sum(z[j] for j=1:n)<=k)

   
    for j=1:n
        @constraint(modelsp, b[j] <= M * z[j])
        @constraint(modelsp, b[j] >= - M * z[j])
    end 
    
    solve(modelsp)
    print(getvalue(t))
    getvalue(b)
end


sparseReg (generic function with 1 method)

In [ ]:
Xmat=Matrix(mysparseDataX)
Ymat=Array(mysparseDataY)
Ymat=Ymat[:] 
typeof(Ymat)
sparseReg(Xmat,Ymat,20,1)
# # 

Academic license - for non-commercial use only
7.11041300374283

200-element Array{Float64,1}:
 -1.0     
 -0.998465
  0.952277
  1.0     
  0.987519
 -0.987328
 -1.0     
  1.0     
 -1.0     
  1.0     
 -0.968356
 -1.0     
 -0.9957  
  ⋮       
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     
  0.0     

In [ ]:
TrueBData

200×1 Array{Int64,2}:
 -1
 -1
  1
  1
  1
 -1
 -1
  1
 -1
  1
 -1
 -1
 -1
  ⋮
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0

In [ ]:
function intersectcount(bstar,btrue)
    count = 0 
    dummy = bstar.*btrue
    l,p = size(dummy)
    for i in 1:l
        if dummy[i] != 0
            count = count + 1
        end
    end
    internonzero=count
    return(internonzero)
end
    

intersectcount (generic function with 1 method)

In [ ]:
function accuracyrate(bstar,btrue,k)
    internonzero = intersectcount(bstar,btrue)
    accrate= internonzero/k * 100 
    return accrate
end

accuracyrate (generic function with 1 method)

In [ ]:
function betastarcount(Bstar,m) 
    changedim = Bstar.*ones(m,1)
    count = 0 
    for i in 1:m
        if (changedim[i] != 0)
            count = count + 1
        end
    end
 return count
end

betastarcount (generic function with 1 method)

In [ ]:
function betatruecount(Btrue,m)
    count = 0 
    for i in 1:m
        if (Btrue[i]!=0)
            count = count + 1 
        end
    end
    return count
end


betatruecount (generic function with 1 method)

In [ ]:
function falsityrate(Bstar, Btrue, k)
    m,n = size(Btrue)
    denom = betatruecount(Btrue,m)
    dummy = Bstar.*Btrue
    changedim=Bstar.*ones(m,1)
    count = 0 
    for i in 1:m 
        if dummy[i] == 0 && Btrue[i] == 0 && changedim[i] != 0
            count = count + 1
        end
    end
    numerator = count
    falsityrate = numerator / denom * 100
    return falsityrate
end

    

falsityrate (generic function with 1 method)

# PROBLEM 2B########################


In [ ]:

tic()
testcaseK20M1 = sparseReg(Xmat,Ymat,20,1)
toc()

tic()
testcaseK20Mhalf = sparseReg(Xmat,Ymat,20,.5)
toc()

tic()
testcaseK20M2 = sparseReg(Xmat,Ymat,20,2)
toc()


Academic license - for non-commercial use only
7.11041300374283elapsed time: 47.834424807 seconds
Academic license - for non-commercial use only
26.759161745592955elapsed time: 4.169224645 seconds
Academic license - for non-commercial use only
6.821958699750449elapsed time: 458.288376103 seconds


458.288376103

In [ ]:
########### TESTCASE K = 20, M = 1 ####################

ARM1 = accuracyrate(testcaseK20M1,TrueBData,20)
FRM1 = falsityrate(testcaseK20M1,TrueBData,20)

############ TEST CASE K = 20, M = .5 #######################

ARMhalf = accuracyrate(testcaseK20Mhalf,TrueBData,20)
FRMhalf = falsityrate(testcaseK20Mhalf, TrueBData,20)

############ TEST CASE K=20, M = 2 ######################

ARM2 = accuracyrate(testcaseK20M2, TrueBData, 20)
FRM2 = falsityrate(testcaseK20M2, TrueBData, 20)


println("ARM1 = ", ARM1)

println("ARM2 = ", ARM2)

println("ARMhalf = ", ARMhalf)

println("FRM1 = ", FRM1)

println("FRM2 = ", FRM2)

println("FRMhalf = ", FRMhalf)



ARM1 = 100.0
ARM2 = 100.0
ARMhalf = 95.0
FRM1 = 0.0
FRM2 = 0.0
FRMhalf = 5.0


In [ ]:
## If M > 1, the accuracy rate -> 100.0, and the false positive rate -> 0. 
## For M = 1,2, the False Positive Rate (FR) = 0.0 and the accuracy rate (AR) = 100.0. 
## If M < 1, the accuracy rate <= 100.0, and the false positive rate >= 0 . For M = .5,
## the False Positive Rate (FR)= 5.0, and the accuracy rate (AR) is  95.0. 
## As M > 1, the run times increase. For M = 1, the run time is  47.834424807
## and for M = 2, the run time is 458.288376103 seconds
## As M < 1, the run times decrease. For M = 1/2, 
## the run time is 4.169224645 seconds.
## If M > 1, the objective values (ie t) get smaller. For M = 1, the t is 7.1104130037428
## and for M = 2, the t is 6.821958699750449. 
## If M < 1, the objective values (ie t) get bigger. For M = .5, the t is 26.759161745592955.

## PROBLEM 2C ###########################################

In [ ]:


Xmat100 = Xmat[1:100,:]
Ymat100 = Ymat[1:100,:]
k = 20
tic()
sparsereg100obsM1 = sparseReg(Xmat100, Ymat100, k, 1)
println(" ")
toc()






Academic license - for non-commercial use only
5.838297180291471 
elapsed time: 4.061509788 seconds


4.061509788

In [ ]:
SP100ARM1 = accuracyrate(sparsereg100obsM1,TrueBData,20)
SP100FRM1 = falsityrate(sparsereg100obsM1,TrueBData,20)
println(SP100ARM1," ", SP100FRM1)


100.0 0.0


In [ ]:
Xmat80 = Xmat[1:80,:]
Ymat80 = Ymat[1:80,:]
k = 20
tic()
sparsereg80obsM1 = sparseReg(Xmat80, Ymat80, k, 1)
println(" ")
toc()


Academic license - for non-commercial use only
5.196960617913158 
elapsed time: 4.920301614 seconds


4.920301614

In [ ]:
SP80ARM1 = accuracyrate(sparsereg80obsM1,TrueBData,20)
SP80FRM1 = falsityrate(sparsereg80obsM1,TrueBData,20)

println(SP80ARM1," ", SP80FRM1)


In [ ]:
#sparsereg60obsM1 = sparseReg(Xmat60, Ymat60, k, 1)                                                
#this errors out and doesn't yield an output. 

In [ ]:
#SP60ARM1 = accuracyrate(sparsereg60obsM1,TrueBData,20)
#SP60FRM1 = falsityrate(sparsereg60obsM1,TrueBData,20)
#Since I am unable to generate an output for the sparse regression, I cannot obtain
# the false positive and accuracy rates. 

In [ ]:
## 
# With M = 1, try running the regression on only the first 100 points in the dataset, then
# the first 80 points only, and finally the first 60 only. What do you observe?

# with 100 and 80 points, the program runs to completion. 
# As you decrease the number of data points, the model takes more time to run.
# For 100 data points, the model takes 4.061509788 seconds to run and yields 
# an objective value of 5.838297180291471. 
# For 80 data points, the model takes 4.920301614 seconds to run and yields
# an objective value of 5.196960617913158. 
# With 60 points, there are not enough points and the model explodes. 
# I was not able to run the model successfully. 


In [3]:
?cat


search: cat catch catalan catch_backtrace catch_stacktrace vcat hcat hvcat



```
cat(dims, A...)
```

Concatenate the input arrays along the specified dimensions in the iterable `dims`. For dimensions not in `dims`, all input arrays should have the same size, which will also be the size of the output array along that dimension. For dimensions in `dims`, the size of the output array is the sum of the sizes of the input arrays along that dimension. If `dims` is a single number, the different arrays are tightly stacked along that dimension. If `dims` is an iterable containing several dimensions, this allows one to construct block diagonal matrices and their higher-dimensional analogues by simultaneously increasing several dimensions for every new input array and putting zero blocks elsewhere. For example, `cat([1,2], matrices...)` builds a block diagonal matrix, i.e. a block matrix with `matrices[1]`, `matrices[2]`, ... as diagonal blocks and matching zero blocks away from the diagonal.
